# LangChain Agent Tutorial: Querying Tabular Data with SQL

This notebook demonstrates how to build a LangChain agent capable of querying tabular data (stored in an Excel file) using SQL. We'll load the data into a Pandas DataFrame, store it in a SQLite database, and then use LangChain's `langgraph` library to create an agent that can:

1.  Understand a natural language question.
2.  Generate a SQL query based on the question and the database schema.
3.  Execute the SQL query against the database.
4.  Generate a natural language answer based on the query result.

## 1. Install Dependencies

First, we need to install the required Python libraries. 

* `pandas`: For data manipulation and reading the Excel file.
* `langchain`: The core LangChain library.
* `langchain-experimental`: Contains experimental LangChain features, including agent toolkits.
* `langchain-openai`: For interacting with OpenAI models (or compatible APIs).
* `langchain-community`: Provides community integrations, including the SQL Database tool.
* `langchain-core`: Core abstractions for LangChain.
* `langgraph`: A library for building stateful, multi-actor applications with LLMs (used here to define the agent's flow).
* `sqlalchemy`: Needed for interacting with the SQL database.
* `openpyxl`: Required by pandas to read `.xlsx` files.
* `typing_extensions`: Provides extended type hinting capabilities.
* `ipython`: Used for displaying images within the notebook.

In [ ]:
!pip install pandas langchain langchain-experimental langchain-openai langchain-community langchain-core langgraph sqlalchemy openpyxl typing_extensions ipython

## 2. Import Initial Libraries

Import the initial set of libraries needed, including pandas for data handling and LangChain components for agents and LLMs.

In [ ]:
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

## 3. Load Data

Load the course information from an Excel file into a Pandas DataFrame. Then, display a random sample of 5 rows to inspect the data structure and content. Ensure the Excel file 'Cleaned_Course_Masterlist_Info.xlsx' is in the same directory as this notebook.

In [ ]:
# Load Data
training_df = pd.read_excel("Cleaned_Course_Masterlist_Info.xlsx")
training_df.sample(5)

## 4. Store Data in SQLite Database

To enable SQL querying by the agent, we store the Pandas DataFrame in a SQLite database.

* Import necessary libraries: `SQLDatabase` from `langchain_community.utilities` and `create_engine` from `sqlalchemy`.
* Create a SQLAlchemy engine connected to a local SQLite database file named `training.db`.
* Use the DataFrame's `to_sql` method to write the data to a table named "training" in the database. `index=False` prevents writing the DataFrame index as a column.

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///training.db")
training_df.to_sql("training", engine, index=False)

## 5. Initialize LangChain SQL Database Connection

Create a LangChain `SQLDatabase` object, wrapping the SQLAlchemy engine. This allows LangChain components to interact with the database.

* Instantiate `SQLDatabase` using the created engine.
* Print the database dialect (e.g., 'sqlite').
* Print the names of tables accessible via this connection (should include 'training').
* Run a sample SQL query directly using `db.run` to verify the connection and retrieve some data.

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM training WHERE SGDCostperpax_25_26 > 1000;"))

## 6. Set Up Language Model (LLM)

Configure the connection to the Language Model. This example uses `ChatOpenAI` but points to a local server URL (`http://127.0.0.1:1235/v1`) and specifies a local model path. 

* Import necessary modules (`getpass`, `os`, `ChatOpenAI`).
* (Commented out) Code to optionally get an API key securely if needed for a cloud-based service.
* Instantiate `ChatOpenAI`, providing the base URL, a placeholder API key (as it's a local server), and the model identifier.

In [ ]:
import getpass
import os

#if not os.environ.get("TOGETHER_API_KEY"):
#  os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter API key for Together AI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://127.0.0.1:1235/v1",
    api_key="token-abc123",
    model=r"/home/otb-02/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct-GPTQ-Int4/snapshots/e9c932ac1893a49ae0fc497ad6e1e86e2e39af20",
)

## 7. Define SQL Query Generation Prompt

Create a prompt template specifically designed to instruct the LLM on how to generate SQL queries based on a user's question and the database schema.

* Import `ChatPromptTemplate`.
* Define a multi-line string template with placeholders for `dialect`, `top_k` (max results), `table_info`, and the user's `input`.
* The template guides the LLM to create syntactically correct SQL for the given dialect, limit results, select relevant columns only, use existing columns/tables, and format the output as JSON with a "query" field.

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

# Convert the text template into a ChatPromptTemplate
query_prompt_template = ChatPromptTemplate.from_template(
    '''
    Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

    Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

    The query should be formatted as JSON with a single field "query":

    {{
      "query": "SELECT COUNT(*) FROM Database;"
    }}

    Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

    Only use the following tables:
    {table_info}

    Question: {input}
    '''
)


## 8. Define Agent State

Define the structure for the agent's state using `TypedDict`. The state will hold information passed between different steps (nodes) in the agent's workflow.

* Import `TypedDict` from `typing_extensions`.
* Define a class `State` inheriting from `TypedDict`.
* Specify the fields the state will contain: `question` (user's input), `query` (generated SQL), `result` (SQL execution output), and `answer` (final natural language response).

In [ ]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

## 9. Define Agent Node: Write Query

Create the first node for the agent graph. This node takes the current state (containing the question) and generates the SQL query.

* Import `Annotated` from `typing_extensions`.
* Define a `QueryOutput` TypedDict to specify the expected JSON structure (a single "query" field) from the LLM for this step.
* Define the `write_query` function that accepts the `State`.
* Inside the function:
    * Invoke the `query_prompt_template` with necessary details (dialect, top_k, table_info, question from the state).
    * Use `llm.with_structured_output(QueryOutput)` to ensure the LLM's response conforms to the `QueryOutput` structure.
    * Invoke the structured LLM with the formatted prompt.
    * Return a dictionary containing the generated query to update the state's "query" field.

In [ ]:
from typing_extensions import Annotated



class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

### Test `write_query` Node

Test the `write_query` function independently by providing a sample state dictionary containing a question.

In [ ]:
write_query({"question": "How many category OSINT courses are there?"})

## 10. Define Agent Node: Execute Query

Create the second node for the agent graph. This node takes the state (containing the generated SQL query) and executes it against the database.

* Import `QuerySQLDatabaseTool` from `langchain_community.tools.sql_database.tool`.
* Define the `execute_query` function that accepts the `State`.
* Inside the function:
    * Instantiate `QuerySQLDatabaseTool`, providing the `db` connection object.
    * Invoke the tool with the SQL query from the state's "query" field.
    * Return a dictionary containing the query result to update the state's "result" field.

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

### Test `execute_query` Node

Test the `execute_query` function independently by providing a sample state dictionary containing a SQL query.

In [ ]:
execute_query({'query': "SELECT COUNT(*) FROM training WHERE Category = 'OSINT';"})

## 11. Define Agent Node: Generate Answer

Create the final node for the agent graph. This node takes the state (containing the original question, the SQL query, and the SQL result) and generates a natural language answer.

* Define the `generate_answer` function that accepts the `State`.
* Inside the function:
    * Construct a prompt string containing the original question, the generated SQL query, and the SQL result as context.
    * Invoke the base LLM (`llm`) with this context prompt.
    * Return a dictionary containing the LLM's response content to update the state's "answer" field.

In [ ]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

## 12. Build Agent Graph

Assemble the defined nodes into a sequential graph using `langgraph`.

* Import `START` and `StateGraph` from `langgraph.graph`.
* Instantiate `StateGraph` with the defined `State` type.
* Use `add_sequence` to define the flow: `write_query` -> `execute_query` -> `generate_answer`.
* Explicitly add an edge from the special `START` node to the first node (`write_query`).
* Compile the graph builder into a runnable `graph` object.

In [ ]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

## 13. Visualize Graph (Optional)

Display a visual representation of the compiled agent graph.

* Import `Image` and `display` from `IPython.display`.
* Call `graph.get_graph().draw_mermaid_png()` to generate a PNG image of the graph.
* Use `display` to show the image in the notebook output.

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## 14. Run the Agent

Execute the compiled graph (the agent) with a specific question.

* Import `pprint` for nicely formatted output.
* Use `graph.stream()` to run the agent. Provide the initial state as a dictionary containing the user's `question`.
* Set `stream_mode="updates"` to get the output from each node as it completes.
* Iterate through the stream and `pprint` each step's output.

In [ ]:
from pprint import pprint

for step in graph.stream(
    {"question": "list me all the courses in the A1-W domain that can teach me about social media data"}, stream_mode="updates"
):
    pprint(step)